In [1]:
#imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.metrics import precision_recall_fscore_support
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import functools
from sklearn.metrics import accuracy_score 
import gensim

In [2]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.4 MB/s eta 0:00:00


In [4]:
import pandas as pd

df = pd.read_csv("depression_dataset_reddit_cleaned.csv",header=0, sep=',',warn_bad_lines=False, error_bad_lines=False)
df_test = pd.read_csv('/content/merged_dataset.csv')
df_test = df_test.drop(['Unnamed: 0'], axis=1)
df_test.head()

<ipython-input-4-15018a8d7b55>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("depression_dataset_reddit_cleaned.csv",header=0, sep=',',warn_bad_lines=False, error_bad_lines=False)
<ipython-input-4-15018a8d7b55>:3: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("depression_dataset_reddit_cleaned.csv",header=0, sep=',',warn_bad_lines=False, error_bad_lines=False)


,content
0,"Don’t worry, it’s just a prop"
1,It never should have happened but that’s exac...
2,"Like really, it’s the depressed kids you’re go..."
3,"Hey there, if you're feeling a bit lost and ov..."
4,no. they are meant to spur demand after elon...


In [5]:
df['clean_text'] = df['clean_text'].str.replace(r'<[^<>]*>', '', regex=True)#1
df['clean_text'] = df['clean_text'].str.replace(r'http://\S+|https://\S+', '', regex=True)#2
df['clean_text']=df['clean_text'].str.lower()
df['clean_text'] = df['clean_text'].str.replace(r'[@_!#$%^&*()<>?/\|}{~:]', '', regex=True)#5
df['clean_text'] = df['clean_text'].replace("\s+", " ", regex=True).str.strip()#last
after_dc=df['clean_text'].str.len().mean()
df_test['clean_text'] = df_test['content'].str.replace(r'<[^<>]*>', '', regex=True)#1
df_test['clean_text'] = df_test['clean_text'].str.replace(r'http://\S+|https://\S+', '', regex=True)#2
df_test['clean_text']=df_test['clean_text'].str.lower()
df_test['clean_text'] = df_test['clean_text'].str.replace(r'[@_!#$%^&*()<>?/\|}{~:]', '', regex=True)#5
df_test['clean_text'] = df_test['clean_text'].replace("\s+", " ", regex=True).str.strip()#last
#after_dc=df['clean_text'].str.len().mean()

#DATA PREPOCESSING
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('wordnet')
nltk.download('omw-1.4')

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')


def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)



# Lemmatizing
df['clean_text'] = df['clean_text'].apply(lambda x: lemmatize_sentence(x))
df_test['clean_text'] = df_test['clean_text'].apply(lambda x: lemmatize_sentence(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [6]:
df_test = df_test.drop(['content'], axis = 1)
df_test.head()

,clean_text
0,"don ’ t worry , it ’ s just a prop"
1,it never should have happen but that ’ s exact...
2,"like really , it ’ s the depressed kid you ’ r..."
3,"hey there , if you 're feel a bit lose and ove..."
4,no . they be mean to spur demand after elon bo...


# BERT

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [7]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=4235836357e1345c5cf81bb57a0c0f7455771deb12de905cbc064e2cbddafa9d
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers


In [8]:
from sentence_transformers import SentenceTransformer
model_bert = SentenceTransformer('all-MiniLM-L6-v2')

In [9]:
X=df['clean_text']
y=df['is_depression']

X_unlabeled = df_test['clean_text']
X_unlabeled.head()

0                   don ’ t worry , it ’ s just a prop
1    it never should have happen but that ’ s exact...
2    like really , it ’ s the depressed kid you ’ r...
3    hey there , if you 're feel a bit lose and ove...
4    no . they be mean to spur demand after elon bo...
Name: clean_text, dtype: object

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [11]:
X_train1 = []
X_unlabeled1 = []

for x in X_train:
    wordveclist = []
    emb = model_bert.encode(x)
    X_train1.append(emb)

for x in X_unlabeled:
    wordveclist = []
    X_unlabeled1.append(model_bert.encode(x)) 

In [12]:
print(len(X_train), len(X_train1))
print(len(X_train1[0]))

6184 6184
384


In [13]:
X_test1 = []
for x in X_test:
    wordveclist = []
    X_test1.append(model_bert.encode(x)) 

In [17]:
class Dataset:
    
    def __init__(self, X, y):
        self.X=torch.as_tensor(X)
        self.y=torch.as_tensor(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        X_1=self.X[index]
        y_1=self.y[index]
        return X_1, y_1

In [18]:
y_train_lst = y_train.values.tolist()

In [19]:
from sklearn.model_selection import train_test_split

#y_train_lst = y_train.values.tolist()
X_train_f, X_val_f, y_train_f, y_val_f = train_test_split(
    X_train1, y_train_lst, test_size=0.2, random_state=0)

In [20]:
#genearting the training set, testing set and the validation set
train_set = Dataset(X_train_f, y_train_f)
valid_set = Dataset(X_val_f, y_val_f)
test_set = Dataset(X_test1, y_test.values)
unlabeled_set = Dataset(X_unlabeled1, np.zeros(len(X_unlabeled1)))
#Generating the data loaders for the training set, testing set and the validation set
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(valid_set,  batch_size=32, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False)
#unlabeled_loader = torch.utils.data.DataLoader(unlabeled_set, shuffle=False)


<ipython-input-17-deb5d617ec55>:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  self.X=torch.as_tensor(X)


In [25]:
#Generating the network architechture
class Net(nn.Module):
    def __init__(self, input_dim=384, output_dim=1, hidden_1=100, hidden_2=10):
        super(Net, self).__init__()
        self.input_dim = input_dim 
        self.output_dim = output_dim
        self.hidden_1 = hidden_1
        self.hidden_2 = hidden_2
        self.fc1 = nn.Linear(self.input_dim, self.hidden_1)
        self.fc2 = nn.Linear(self.hidden_1, self.hidden_2)
        self.fc3 = nn.Linear(self.hidden_2, self.output_dim)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

In [26]:
#Calling the Net is_depression to initialize the neural network
model_te = Net()
print(model_te)

Net(
  (fc1): Linear(in_features=384, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
  (fc3): Linear(in_features=10, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [27]:
#Defining the loss and the optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model_te.parameters(), lr=0.01)

In [28]:
valid_loss_min = np.Inf
max_epochs=45

for epoch in range(max_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    model_te.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model_te(data)

        #print(output)
        #print(target)
        target = target.unsqueeze(1)
        target = target.float()
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
    model_te.eval()
    for data, target in valid_loader:
        output = model_te(data)
        target = target.unsqueeze(1)
        target = target.float()
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    #Printing the loss values
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model_te.state_dict(), 'model_te.pt')
        valid_loss_min = valid_loss


Epoch: 1 	Training Loss: 0.697409 	Validation Loss: 0.697187
Validation loss decreased (inf --> 0.697187).  Saving model ...
Epoch: 2 	Training Loss: 0.694541 	Validation Loss: 0.694728
Validation loss decreased (0.697187 --> 0.694728).  Saving model ...
Epoch: 3 	Training Loss: 0.693205 	Validation Loss: 0.693376
Validation loss decreased (0.694728 --> 0.693376).  Saving model ...
Epoch: 4 	Training Loss: 0.692335 	Validation Loss: 0.692468
Validation loss decreased (0.693376 --> 0.692468).  Saving model ...
Epoch: 5 	Training Loss: 0.691687 	Validation Loss: 0.691637
Validation loss decreased (0.692468 --> 0.691637).  Saving model ...
Epoch: 6 	Training Loss: 0.691095 	Validation Loss: 0.690872
Validation loss decreased (0.691637 --> 0.690872).  Saving model ...
Epoch: 7 	Training Loss: 0.690374 	Validation Loss: 0.690053
Validation loss decreased (0.690872 --> 0.690053).  Saving model ...
Epoch: 8 	Training Loss: 0.689706 	Validation Loss: 0.689097
Validation loss decreased (0.69005

In [29]:
#Function to test the model
def predict(model, dataloader):
    prediction_list = []
    truth_list = []
    m = 0
    cnt = 0
    for i, batch in enumerate(dataloader):
        outputs = model(batch[0])
        if(outputs[0] < 0.5):
          predicted = 0
        else:
          predicted = 1
        #print(batch[1], predicted)
        if(int(batch[1].item()) == int(predicted)):
          m+=1
        cnt+=1
        #_, predicted = torch.max(outputs.data, 1)
        prediction_list.append(int(predicted))
        truth_list.append(int(batch[1].item()))
    #acc = m/cnt
    return prediction_list, truth_list

In [30]:
from sklearn.metrics import classification_report

#Loading the model and generating the predictions
model_te.load_state_dict(torch.load('model_te.pt'))
pred, true_val = predict(model_te, test_loader)

print(classification_report(true_val, pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       779
           1       0.97      0.95      0.96       768

    accuracy                           0.96      1547
   macro avg       0.96      0.96      0.96      1547
weighted avg       0.96      0.96      0.96      1547



In [ ]:
#Calling the Net is_depression to initialize the neural network
model_te = Net()
print(model_te)

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model_te.parameters(), lr=0.01)

In [31]:
#traning and validating the model
valid_loss_min = np.Inf
max_epochs=45
flag = 0
while(flag == 0):
  train_set = Dataset(X_train_f, y_train_f)
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True, num_workers=0)
  unlabeled_set = Dataset(X_unlabeled1, np.zeros(len(X_unlabeled1)))
  unlabeled_loader = torch.utils.data.DataLoader(unlabeled_set, shuffle=False)
  for epoch in range(max_epochs):
      train_loss = 0.0
      valid_loss = 0.0
      model_te.train()
      for data, target in train_loader:
          optimizer.zero_grad()
          output = model_te(data)

          #print(output)
          #print(target)
          target = target.unsqueeze(1)
          target = target.float()
          loss = criterion(output, target)
          loss.backward()
          optimizer.step()
          train_loss += loss.item()*data.size(0)
      model_te.eval()
      for data, target in valid_loader:
          output = model_te(data)
          target = target.unsqueeze(1)
          target = target.float()
          loss = criterion(output, target)
          valid_loss += loss.item()*data.size(0)
      train_loss = train_loss/len(train_loader.dataset)
      valid_loss = valid_loss/len(valid_loader.dataset)
      #Printing the loss values
      print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch+1, 
          train_loss,
          valid_loss
          ))
      if valid_loss <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min,
          valid_loss))
          torch.save(model_te.state_dict(), 'model_te.pt')
          valid_loss_min = valid_loss
  
  flag = 1
  model_te.load_state_dict(torch.load('model_te.pt'))
  k=0
  products_list = ['laptop', 'printer', 'tablet', 'desk', 'chair']

  df_unlabeled = pd.DataFrame(zip(X_unlabeled1), columns = ['text'])
  drp = []
  print(len(X_train_f),len(y_train_f), len(X_unlabeled1))
  for i ,batch in enumerate(unlabeled_loader):
    outputs = model_te(batch[0][0])
    #print(len(batch[0][0]))
    #print(outputs)
    if(outputs[0]<= 0.1 or outputs[0] > 0.9):
      flag = 0
      X_train_f.append(batch[0][0])
      if(outputs[0] < 0.5):
        y_train_f.append(0)
      else:
        y_train_f.append(1)
      drp.append(i)
    
        #X_train_f.append(batch)
  
    #print(outputs[1])
    #break
  df_unlabeled = df_unlabeled.drop(drp)
  X_unlabeled1 = df_unlabeled['text'].values.tolist()
  print(len(X_train_f),len(y_train_f), len(X_unlabeled1))
  #print(X_unlabeled1[0])
  #print(type(X_unlabeled1[0]))
  #print(type(X_unlabeled1))

Epoch: 1 	Training Loss: 0.165470 	Validation Loss: 0.143551
Validation loss decreased (inf --> 0.143551).  Saving model ...
Epoch: 2 	Training Loss: 0.162305 	Validation Loss: 0.141209
Validation loss decreased (0.143551 --> 0.141209).  Saving model ...
Epoch: 3 	Training Loss: 0.159807 	Validation Loss: 0.139020
Validation loss decreased (0.141209 --> 0.139020).  Saving model ...
Epoch: 4 	Training Loss: 0.153174 	Validation Loss: 0.137117
Validation loss decreased (0.139020 --> 0.137117).  Saving model ...
Epoch: 5 	Training Loss: 0.151195 	Validation Loss: 0.136053
Validation loss decreased (0.137117 --> 0.136053).  Saving model ...
Epoch: 6 	Training Loss: 0.152805 	Validation Loss: 0.132852
Validation loss decreased (0.136053 --> 0.132852).  Saving model ...
Epoch: 7 	Training Loss: 0.148245 	Validation Loss: 0.131620
Validation loss decreased (0.132852 --> 0.131620).  Saving model ...
Epoch: 8 	Training Loss: 0.148182 	Validation Loss: 0.129603
Validation loss decreased (0.13162

In [32]:
from sklearn.metrics import classification_report

#Loading the model and generating the predictions
model_te.load_state_dict(torch.load('model_te.pt'))
pred, true_val = predict(model_te, test_loader)

print(classification_report(true_val, pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       779
           1       0.97      0.94      0.96       768

    accuracy                           0.96      1547
   macro avg       0.96      0.96      0.96      1547
weighted avg       0.96      0.96      0.96      1547

